In [11]:
# ============================================
# CLIENTE INTEGRAL - DATOS DE EJEMPLO (CORREGIDO)
# ============================================

import pandas as pd
import numpy as np
import random

# -----------------------------
# 1) Base de 50 clientes con 3 áreas
# -----------------------------
np.random.seed(42)
random.seed(42)

clientes = [f"C{str(i).zfill(3)}" for i in range(1, 51)]

df_clientes = pd.DataFrame({
    "id_cliente": clientes,
    "brilla": np.random.randint(0, 2, 50),
    "consumo": np.random.randint(0, 2, 50),
    "sad": np.random.randint(0, 2, 50),
})

print("✅ Base de clientes (pertenencia por área):")
print(df_clientes.head())

# Guarda en la carpeta actual
df_clientes.to_csv("base_clientes.csv", index=False)
print("💾 Guardado: base_clientes.csv")


# -----------------------------
# 2) Generar dimensiones SOLO para clientes con 1 en el área
#    (y elegir un subconjunto aleatorio de esos elegibles)
# -----------------------------
def generar_dimensiones_area(df_base, area, min_frac=0.6, max_frac=0.9):
    """
    - Filtra clientes con 1 en el área.
    - Toma un subconjunto aleatorio (entre 60% y 90% por defecto) de esos elegibles.
    - Genera dimensiones en [0,1] (aquí entre 0.4 y 1.0 para evitar valores muy bajos).
    """
    elegibles = df_base.loc[df_base[area] == 1, "id_cliente"].tolist()
    if not elegibles:
        # Si no hay elegibles para esa área, retorna un DataFrame vacío
        return pd.DataFrame(columns=[
            "id_cliente", "dimension_economica", "dimension_relacional",
            "dimension_cumplimiento", "dimension_potencial"
        ])

    # Tamaño del subconjunto aleatorio
    k_min = max(1, int(len(elegibles) * min_frac))
    k_max = max(k_min, int(len(elegibles) * max_frac))
    k = random.randint(k_min, k_max)

    subset = random.sample(elegibles, k)

    df_dim = pd.DataFrame({
        "id_cliente": subset,
        "dimension_economica": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_relacional": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_cumplimiento": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
        "dimension_potencial": np.round(np.random.uniform(0.4, 1.0, len(subset)), 2),
    })

    df_dim.to_csv(f"dimensiones_{area}.csv", index=False)
    print(f"💾 Guardado: dimensiones_{area}.csv (clientes elegibles: {len(elegibles)}, usados: {len(subset)})")

    return df_dim

# Crear dimensiones por área respetando la pertenencia
df_brilla  = generar_dimensiones_area(df_clientes, "brilla")
df_consumo = generar_dimensiones_area(df_clientes, "consumo")
df_sad     = generar_dimensiones_area(df_clientes, "sad")


# -----------------------------
# 3) (Opcional) Consolidado de dimensiones
# -----------------------------
df_dimensiones = pd.concat([
    df_brilla.assign(area="brilla"),
    df_consumo.assign(area="consumo"),
    df_sad.assign(area="sad"),
], ignore_index=True)

df_dimensiones.to_csv("dimensiones_todas.csv", index=False)
print("💾 Guardado: dimensiones_todas.csv")


# -----------------------------
# 4) (Opcional) Vista rápida de consistencia
#    - Ningún id en dimensiones_{area} debe tener 0 en esa área en df_clientes
# -----------------------------
def chequear_consistencia(df_base, df_dim, area):
    if df_dim.empty:
        print(f"🔎 {area}: sin filas (no había elegibles).")
        return
    merged = df_dim.merge(df_base[["id_cliente", area]], on="id_cliente", how="left")
    inconsistentes = merged.loc[merged[area] != 1]
    if inconsistentes.empty:
        print(f"✅ Consistencia {area}: OK (solo clientes con 1).")
    else:
        print(f"⚠️ Inconsistencia {area}: hay clientes con 0 en el área en la tabla de dimensiones.")
        print(inconsistentes)

chequear_consistencia(df_clientes, df_brilla,  "brilla")
chequear_consistencia(df_clientes, df_consumo, "consumo")
chequear_consistencia(df_clientes, df_sad,     "sad")

✅ Base de clientes (pertenencia por área):
  id_cliente  brilla  consumo  sad
0       C001       0        0    0
1       C002       1        1    1
2       C003       0        0    1
3       C004       0        1    1
4       C005       0        0    1
💾 Guardado: base_clientes.csv
💾 Guardado: dimensiones_brilla.csv (clientes elegibles: 27, usados: 17)
💾 Guardado: dimensiones_consumo.csv (clientes elegibles: 29, usados: 17)
💾 Guardado: dimensiones_sad.csv (clientes elegibles: 26, usados: 18)
💾 Guardado: dimensiones_todas.csv
✅ Consistencia brilla: OK (solo clientes con 1).
✅ Consistencia consumo: OK (solo clientes con 1).
✅ Consistencia sad: OK (solo clientes con 1).
